과제: 파이썬 머신러닝 완벽 가이드 ch8. 7 

마감: 4월 29일(월) 18:30

pg. 516~528를 필사하여 깃허브 주소를 댓글로 남겨주세요.

​

필요한 데이터를 다운받는 주소가 다음으로 바뀌었으니 참고 부탁드립니다.

https://archive.ics.uci.edu/dataset/191/opinosis+opinion+frasl+review

# Chap08. 텍스트 분석

## 08-07. 문서 군집화 소개와 실습 (Opinion Review 데이터 세트)

-------
### 문서 군집화 개념 (Document Clustering)

문서 군집화 : 비슷한 텍스트 구성의 문서를 군집화 하는 것.
- 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 텍스트 분류 기반의 문서 분류와 유사하다.
- 텍스트 분류 기반 문서 분류 : 사전에 결정 카테고리 값을 지닌 학습 데이터 세트가 필요하다
- 문서 군집화 : 학습 데이터 세트가 필요없는 비지도학습 기반으로 동작한다.

-> 이전에 배운 군집화 기법을 활용해 텍스트 기반의 문서 군집화를 적용해보자

--------
### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기
51개의 파일이 있으며 이는 자동차나 아이팟과 같은 전자 제품이나 호텔 서비스 등에 대한 리뷰 내용을 담고 있다. 여기서는 여러 개의 파일을 한 개의 DataFrame으로 로딩하여 데이터 처리를 한다. 파일들을 하나씩 읽어서 파일 명과 파일 리뷰를 하나의 DF로 로드하여 파일명 별로 어떤 리뷰를 담고 있는지 개략적으로 살펴보자.

In [2]:
import pandas as pd
import glob, os

# 다음은 저자의 컴퓨터에서 압축 파일을 풀어 놓은 디렉터리이니, 각자 디렉터리를 다시 설정한다.
path = r'C:\workspace\essa\assignment\24-1\data\OpinosisDataset1.0\topics'

# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합.
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list로 취합한다.
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합한다.
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col = None, header=0, encoding = 'latin1')

    # 절대 경로로 주어진 파일명을 가공한다
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename' : filename_list, 'opinion_text' : opinion_text})
document_df.head()

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


파일 이름 자체만으로 의견의 텍스트가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있다.

-----------

<문서를 TF-IDF 형태로 벡터화>
tokenizer : Lemmatization을 구현한 LemNormalize() 함수
- ngram : (1, 2)
- min_df, max_df 범위를 설정하여 피처 개수 제한

In [6]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\홍예원\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', 
                             ngram_range=(1, 2), min_df = 0.05, max_df=0.85)

# opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


- 문서 유형 : 전자제품, 자동차, 호텔

5개의 중심 (Centroid) 기반으로 어떻게 군집화되는지 확인해보자

In [8]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


cluster_label 칼럼을 추가 : 각 데이터별로 할당된군집의 레이블을 파일명과 파일 내용을 가지고 있는 document_df에 추가

In [9]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,4
1,bathroom_bestwestern_hotel_sfo,...,3
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


In [10]:
# document_df에서 cluster_label로 어떤 파일명으로 매칭되었는지 보면서 군집화 결과 확인

# cluster #0 : 호텔에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==0].sort_values(by = 'filename')

,filename,opinion_text,cluster_label
17,interior_honda_accord_2008,...,0
18,interior_toyota_camry_2007,...,0


In [11]:
# cluster #1 : 전자기기에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==1].sort_values(by = 'filename') 

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
35,screen_ipod_nano_8gb,...,1
36,screen_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [12]:
# cluster #2 : 차량용 내비게이션에 대한 리뷰로 군집화
# 그러나 cluster #1과 비슷하게 킨들, 아이팟, 넷북이 포함되어 있음
document_df[document_df['cluster_label']==2].sort_values(by = 'filename') 

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,...,2
7,comfort_toyota_camry_2007,...,2
16,gas_mileage_toyota_camry_2007,...,2
22,mileage_honda_accord_2008,...,2
25,performance_honda_accord_2008,...,2
27,price_amazon_kindle,...,2
29,quality_toyota_camry_2007,...,2
37,seats_honda_accord_2008,...,2
47,transmission_toyota_camry_2007,...,2


In [13]:
# cluster #3 : 호텔에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==3].sort_values(by = 'filename') 

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,3
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
28,price_holiday_inn_london,...,3
32,room_holiday_inn_london,...,3
30,rooms_bestwestern_hotel_sfo,...,3


In [14]:
# cluster #4 : 자동차에 대한 리뷰로 군집화
document_df[document_df['cluster_label']==4].sort_values(by = 'filename') 

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,4
5,buttons_amazon_kindle,...,4
8,directions_garmin_nuvi_255W_gps,...,4
9,display_garmin_nuvi_255W_gps,...,4
10,eyesight-issues_amazon_kindle,...,4
11,features_windows7,...,4
12,fonts_amazon_kindle,...,4
23,navigation_amazon_kindle,...,4
33,satellite_garmin_nuvi_255W_gps,...,4
34,screen_garmin_nuvi_255W_gps,...,4


전반적으로 군집화 결과를 보면 군집 개수가 약간 많게 설정되어 군집이 세분화된 경향이 있다.

중심 개수를 5개에서 3개로 낮추어 군집화해보자.

In [15]:
from sklearn.cluster import KMeans

# 3개 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

# 소숙 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by = 'cluster_label')

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,filename,opinion_text,cluster_label
24,parking_bestwestern_hotel_sfo,...,0
21,location_holiday_inn_london,...,0
32,room_holiday_inn_london,...,0
15,free_bestwestern_hotel_sfo,...,0
14,food_swissotel_chicago,...,0
13,food_holiday_inn_london,...,0
40,service_swissotel_hotel_chicago,...,0
38,service_bestwestern_hotel_sfo,...,0
20,location_bestwestern_hotel_sfo,...,0
39,service_holiday_inn_london,...,0


--------
### 군집별 핵심 단어 추출하기

각 군집에 속한 문서는 핵심 단어를 주축으로 군집화 되어있을 것이다. 이번에는 각 군집을 구성하는 핵심 단어를 확인해보자.

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심 (Centroid)을 중심으로 얼마나 가깝게 위치해있는지 clustser_centers 라는 속성으로 제공한다.

- cluster_centers_는 배열값으로 제공
- 행 : 개별 군집
- 열 : 개별 피처
- 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표값

In [16]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape : ', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape :  (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


군집이 3개, word 피처가 2409개로 구성되어 있다.

- 각 행의 배열 값은 각 군집 내의 2409개의 피처의 위치가 개별 중심과 얼마나 가까운가를 상대값으로 나타낸 것.
- 0~1까지의 값을 가질 수 있다.
- 1에 가까울 수록 중심과 가까운 값임을 의미

------ 
cluster_centers 속성값을 이용해 각 군집별 핵심 단어를 찾아보자
- cluster_centers : numpy의 ndarray
- ndarray의 argsort()[:, ::-1]을 이용하여 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값을 반환한다. -> 핵심 단어 피처의 이름을 출력하기 위해서
- get_cluster_details() : cluster_centers_ 배열 내 값이 가장 큰 데이터의 위치 인덱스를 추출한 뒤, 해당 인덱스를 이용해 핵심 단어 이름과 그때의 상대 위치 값을 추출해 cluster_details 라는 Dict 객체 변수에 기록하고 반환한다.

In [17]:
# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 반환

def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details = {}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화.
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구한다.
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값을 구한다.
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심 위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

< 개별 군집 번호, 핵심 단어, 핵심 단어 중심 위치 상댓값, 파일명 속성값 정보 > 등을 좀 더 보기좋게 표현하기 위해 별도의 함수를 생성해보자.

In [18]:
# <개별 군집 번호, 핵심 단어, 핵심 단어 중심 위치 상댓값, 파일명 속성값 정보> 출력하는 함수
def print_cluster_details(clsuter_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features : ', cluster_detail['top_features'])
    print('Reviews 파일명 : ', cluster_detail['filenames'][:7])
    print('================================================')


In [19]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num=3, top_n_features = 10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features :  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 :  ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
###### Cluster 1
Top features :  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 :  ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
###### Cluster 2
Top features :  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 :  ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery